<a href="https://colab.research.google.com/github/dawid258/POSTPROCESS/blob/main/Mozaikowanie_z_filtrem_Gaussa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 29.9 MB/s eta 0:00:00


#Przytnij raster

In [ ]:
# 1. Zainstaluj GDAL
!pip install GDAL

# 2. Zaimportuj i przytnij
from osgeo import gdal

input_raster  = '/content/pila_mozaika.tif'
mask_vector   = '/content/m_Pila_epsg_2180.gpkg'
output_raster = '/content/Pila_solar_cut.tif'

# Wywołanie przycięcia
gdal.Warp(
    output_raster,
    input_raster,
    cutlineDSName=mask_vector,
    cropToCutline=True
)

print("Zapisano:", output_raster)


/usr/local/lib/python3.11/dist-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Zapisano: /content/Pila_solar_cut.tif


In [ ]:
# 1. Zainstaluj GDAL
!pip install GDAL

# 2. Zaimportuj i przytnij
from osgeo import gdal

input_raster  = '/content/Pila_solar_cut.tif'
mask_vector   = '/content/budynki_v2_roof_height_v2.gpkg'
output_raster = '/content/Pila_solar_buldings.tif'

# Wywołanie przycięcia
gdal.Warp(
    output_raster,
    input_raster,
    cutlineDSName=mask_vector,
    cropToCutline=True
)

print("Zapisano:", output_raster)


/usr/local/lib/python3.11/dist-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Zapisano: /content/Pila_solar_buldings.tif


#Klasyfikacja

#gauss min

In [ ]:
import os
import glob
import numpy as np
import rasterio
from rasterio.transform import from_origin
import tempfile
import shutil
from tqdm import tqdm
import psutil
import gc

def mosaic_with_gauss_grid(input_folder: str,
                           output_path: str,
                           feather_px: int = 20,
                           tile_size: int = 2048,
                           cleanup_temp: bool = True):
    """
    Mozaikowanie kafelków z wyborem niższych wartości i gaussowskim blendowaniem krawędzi
    z wykorzystaniem plików tymczasowych dla efektywnego zarządzania pamięcią.

    Args:
        input_folder: Folder z plikami .tif
        output_path: Ścieżka do pliku wyjściowego
        feather_px: Rozmiar strefy blendowania w pikselach
        tile_size: Rozmiar kafelków do przetwarzania (dla zarządzania pamięcią)
        cleanup_temp: Czy usunąć pliki tymczasowe po zakończeniu
    """

    # Stworzenie katalogu tymczasowego
    temp_dir = tempfile.mkdtemp(prefix='mosaic_temp_')
    print(f"Katalog tymczasowy: {temp_dir}")

    try:
        # 1. Lista plików i parametry pierwszego kafla
        files = sorted(glob.glob(os.path.join(input_folder, '*.tif')))
        if not files:
            raise ValueError("Brak plików .tif w katalogu")

        print(f"Znaleziono {len(files)} plików do przetworzenia")

        with rasterio.open(files[0]) as src0:
            crs    = src0.crs
            dtype  = src0.dtypes[0]
            res    = src0.res[0]
            nodata = src0.nodata
            b0     = src0.bounds

        # 2. Wyznacz globalne granice bez marginesu
        print("Wyznaczanie globalnych granic...")
        x_min, x_max = b0.left,  b0.right
        y_min, y_max = b0.bottom, b0.top

        for fn in tqdm(files[1:], desc="Analizowanie granic"):
            with rasterio.open(fn) as src:
                b = src.bounds
                x_min, x_max = min(x_min, b.left),  max(x_max, b.right)
                y_min, y_max = min(y_min, b.bottom), max(y_max, b.top)

        # 3. Stwórz bufor o wymiarach +2*feather_px
        width  = int(np.ceil((x_max - x_min) / res)) + 2 * feather_px
        height = int(np.ceil((y_max - y_min) / res)) + 2 * feather_px
        transform = from_origin(
            x_min - feather_px * res,
            y_max + feather_px * res,
            res, res
        )

        print(f"Wymiary końcowej mozaiki: {width}x{height} pikseli")
        print(f"Szacowany rozmiar: {width * height * 8 / (1024**3):.2f} GB (float64)")

        # Tworzenie plików tymczasowych dla minimum, wag i pomocniczych
        min_values_path = os.path.join(temp_dir, 'min_values.dat')
        blend_weights_path = os.path.join(temp_dir, 'blend_weights.dat')
        has_data_path = os.path.join(temp_dir, 'has_data.dat')

        # Inicjalizacja plików tymczasowych
        print("Inicjalizacja plików tymczasowych...")
        _initialize_temp_files_min(min_values_path, blend_weights_path, has_data_path, height, width)

        # parametry gaussa
        sigma = feather_px / 2.0

        # 4. Przetwarzaj każdy kafel
        print("Przetwarzanie kafelków...")
        for i, fn in enumerate(tqdm(files, desc="Przetwarzanie kafelków")):
            _process_tile_min(fn, min_values_path, blend_weights_path, has_data_path,
                         height, width, nodata, feather_px, sigma,
                         x_min, y_max, res, tile_size)

            # Monitorowanie pamięci
            if i % 10 == 0:
                memory_percent = psutil.virtual_memory().percent
                print(f"Użycie pamięci: {memory_percent:.1f}%")
                if memory_percent > 85:
                    print("Wysoki poziom używania pamięci - wymuszanie garbage collection")
                    gc.collect()

        # 5. Finalna mozaika - przetwarzanie kafelkami
        print("Tworzenie finalnej mozaiki z blendowaniem...")
        _create_final_mosaic_min(min_values_path, blend_weights_path, has_data_path,
                           output_path, height, width, dtype, nodata,
                           crs, transform, tile_size, feather_px, sigma)

        print(f"Mozaika zapisana: {output_path}")
        print(f"Granice mozaiki: {x_min}, {y_min}, {x_max}, {y_max}")

    finally:
        # Sprzątanie plików tymczasowych
        if cleanup_temp:
            print("Usuwanie plików tymczasowych...")
            shutil.rmtree(temp_dir)
        else:
            print(f"Pliki tymczasowe zachowane w: {temp_dir}")


def _initialize_temp_files_min(min_values_path: str, blend_weights_path: str,
                              has_data_path: str, height: int, width: int):
    """Inicjalizacja plików tymczasowych z wartościami początkowymi"""

    # Tworzenie plików jako memory-mapped arrays
    min_values = np.memmap(min_values_path, dtype='float64', mode='w+',
                          shape=(height, width))
    blend_weights = np.memmap(blend_weights_path, dtype='float64', mode='w+',
                             shape=(height, width))
    has_data = np.memmap(has_data_path, dtype='bool', mode='w+',
                        shape=(height, width))

    # Inicjalizacja w blokach
    block_size = 1024
    for i in tqdm(range(0, height, block_size), desc="Inicjalizacja plików temp"):
        end_i = min(i + block_size, height)
        min_values[i:end_i, :] = np.inf  # Inicjalizacja z nieskończonością dla minimum
        blend_weights[i:end_i, :] = 0.0
        has_data[i:end_i, :] = False

    # Wymuszenie zapisu na dysk
    del min_values, blend_weights, has_data


def _process_tile_min(fn: str, min_values_path: str, blend_weights_path: str,
                     has_data_path: str, height: int, width: int, nodata: float,
                     feather_px: int, sigma: float, x_min: float, y_max: float,
                     res: float, tile_size: int):
    """Przetwarzanie pojedynczego kafla z wyborem minimum"""

    with rasterio.open(fn) as src:
        arr = src.read(1).astype('float64')
        mask_nodata = (arr == nodata)

        h, w = src.shape

        # odległość do najbliższej krawędzi kafla
        yy = np.arange(h).reshape(h, 1)
        xx = np.arange(w).reshape(1, w)
        dist_x = np.minimum(xx, w - 1 - xx)
        dist_y = np.minimum(yy, h - 1 - yy)
        dist   = np.minimum(dist_x, dist_y).astype('float64')

        # macierz wag dla blendowania (tylko na krawędziach)
        w_mat = np.ones_like(dist)
        if feather_px > 0:
            feather_zone = (dist < feather_px)
            # Liniowy spadek wagi od 1.0 do 0.0 w strefie blendowania
            w_mat[feather_zone] = dist[feather_zone] / feather_px

        # Ustawienie wag na 0 dla nodata
        w_mat[mask_nodata] = 0.0

        # Ustawienie wartości na infinity dla nodata (nie wpływa na minimum)
        arr[mask_nodata] = np.inf

        # offsety w buforze (w pikselach)
        col_off = int((src.bounds.left  - x_min) / res) + feather_px
        row_off = int((y_max - src.bounds.top) / res) + feather_px
        r0, r1 = row_off, row_off + h
        c0, c1 = col_off, col_off + w

        # Otwarcie plików tymczasowych
        min_values = np.memmap(min_values_path, dtype='float64', mode='r+',
                              shape=(height, width))
        blend_weights = np.memmap(blend_weights_path, dtype='float64', mode='r+',
                                 shape=(height, width))
        has_data = np.memmap(has_data_path, dtype='bool', mode='r+',
                            shape=(height, width))

        # Akumulacja w blokach
        block_h = min(tile_size, h)
        block_w = min(tile_size, w)

        for i in range(0, h, block_h):
            for j in range(0, w, block_w):
                end_i = min(i + block_h, h)
                end_j = min(j + block_w, w)

                arr_block = arr[i:end_i, j:end_j]
                w_block = w_mat[i:end_i, j:end_j]
                valid_block = ~mask_nodata[i:end_i, j:end_j]

                r0_block = r0 + i
                r1_block = r0 + end_i
                c0_block = c0 + j
                c1_block = c0 + end_j

                # Tylko dla pikseli z wartymi danymi
                if np.any(valid_block):
                    # Aktualizacja minimum
                    current_min = min_values[r0_block:r1_block, c0_block:c1_block]
                    new_min = np.minimum(current_min, arr_block)
                    min_values[r0_block:r1_block, c0_block:c1_block] = new_min

                    # Akumulacja wag dla blendowania
                    blend_weights[r0_block:r1_block, c0_block:c1_block] += w_block * valid_block

                    # Oznaczenie obszarów z danymi
                    has_data[r0_block:r1_block, c0_block:c1_block] |= valid_block

        # Czyszczenie referencji
        del min_values, blend_weights, has_data, arr, w_mat


def _create_final_mosaic_min(min_values_path: str, blend_weights_path: str,
                            has_data_path: str, output_path: str, height: int, width: int,
                            dtype: str, nodata: float, crs, transform,
                            tile_size: int, feather_px: int, sigma: float):
    """Tworzenie finalnej mozaiki z liniowym blendowaniem Gaussa"""

    # Otwarcie plików tymczasowych
    min_values = np.memmap(min_values_path, dtype='float64', mode='r',
                          shape=(height, width))
    blend_weights = np.memmap(blend_weights_path, dtype='float64', mode='r',
                             shape=(height, width))
    has_data = np.memmap(has_data_path, dtype='bool', mode='r',
                        shape=(height, width))

    # Profil dla pliku wyjściowego
    profile = {
        'driver': 'GTiff',
        'dtype': dtype,
        'count': 1,
        'crs': crs,
        'transform': transform,
        'width': width,
        'height': height,
        'nodata': nodata,
        'compress': 'lzw',
        'tiled': True,
        'blockxsize': 512,
        'blockysize': 512
    }

    with rasterio.open(output_path, 'w', **profile) as dst:
        # Przetwarzanie w blokach
        for i in tqdm(range(0, height, tile_size), desc="Zapisywanie mozaiki"):
            end_i = min(i + tile_size, height)

            for j in range(0, width, tile_size):
                end_j = min(j + tile_size, width)

                # Wczytanie bloków
                min_block = min_values[i:end_i, j:end_j]
                weights_block = blend_weights[i:end_i, j:end_j]
                data_block = has_data[i:end_i, j:end_j]

                # Przygotowanie wyniku
                result_block = np.full((end_i - i, end_j - j), nodata, dtype='float64')

                # Obszary z danymi
                valid = data_block & (min_block != np.inf)

                if np.any(valid):
                    # Podstawowe wartości minimum
                    result_block[valid] = min_block[valid]

                    # Zastosowanie blendowania Gaussa na krawędziach
                    # Tworzenie maski krawędzi (obszary z wagami < 1.0)
                    edge_mask = valid & (weights_block < 0.99) & (weights_block > 0.01)

                    if np.any(edge_mask):
                        # Liniowe blendowanie z filtrem Gaussa
                        gauss_weights = np.exp(-((1.0 - weights_block[edge_mask])**2) / (2 * (sigma/feather_px)**2))

                        # Płynne przejście między wartościami
                        result_block[edge_mask] = (
                            min_block[edge_mask] * gauss_weights +
                            result_block[edge_mask] * (1 - gauss_weights)
                        )

                # Zapis bloku
                dst.write(result_block.astype(dtype), 1,
                         window=rasterio.windows.Window(j, i, end_j - j, end_i - i))

    # Czyszczenie
    del min_values, blend_weights, has_data


def get_memory_usage():
    """Zwraca aktualne użycie pamięci w GB"""
    return psutil.virtual_memory().used / (1024**3)


def estimate_memory_requirements(input_folder: str, feather_px: int = 20):
    """Szacuje wymagania pamięciowe dla mozaiki"""

    files = sorted(glob.glob(os.path.join(input_folder, '*.tif')))
    if not files:
        return 0

    with rasterio.open(files[0]) as src0:
        res = src0.res[0]
        b0 = src0.bounds

    x_min, x_max = b0.left, b0.right
    y_min, y_max = b0.bottom, b0.top

    for fn in files[1:]:
        with rasterio.open(fn) as src:
            b = src.bounds
            x_min, x_max = min(x_min, b.left), max(x_max, b.right)
            y_min, y_max = min(y_min, b.bottom), max(y_max, b.top)

    width = int(np.ceil((x_max - x_min) / res)) + 2 * feather_px
    height = int(np.ceil((y_max - y_min) / res)) + 2 * feather_px

    # Szacowanie rozmiaru (3 macierze: 2x float64 + 1x bool + bufor)
    size_gb = (width * height * (8 + 8 + 1)) / (1024**3)

    return size_gb, width, height


def clip_raster_to_mask(raster_path, mask_path, layer=None, output_path=None):
    """
    Przytnie rastr do obszaru zdefiniowanego w pliku .gpkg.

    :param raster_path: ścieżka do wejściowego pliku .tif
    :param mask_path: ścieżka do pliku .gpkg z maską
    :param layer: (opcjonalnie) nazwa warstwy w .gpkg; jeśli None, używa pierwszej
    :param output_path: (opcjonalnie) ścieżka do zapisu wynikowego .tif
    :return: ścieżka do przyciętego rastra
    """
    import geopandas as gpd
    from rasterio.mask import mask

    # wczytaj maskę
    mask_gdf = gpd.read_file(mask_path, layer=layer)
    geoms = mask_gdf.geometry.values

    # otwórz rastr i przytnij
    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, geoms, crop=True)
        out_meta = src.meta.copy()

    # zaktualizuj metadane
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width":  out_image.shape[2],
        "transform": out_transform
    })

    # wyznacz ścieżkę wyjściową, jeśli nie podano
    if output_path is None:
        output_path = raster_path.replace(".tif", "_clipped.tif")

    # zapisz przycięty rastr
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(out_image)

    return output_path


input_folder = r'/content/tile_outdoor_comfort'
output_file = 'Pila_outdoor_comfort.tif'

# Szacowanie wymagań pamięciowych
print("Szacowanie wymagań pamięciowych...")
memory_gb, w, h = estimate_memory_requirements(input_folder, feather_px=20)
print(f"Szacowany rozmiar danych: {memory_gb:.2f} GB")
print(f"Wymiary: {w}x{h} pikseli")

available_memory = psutil.virtual_memory().available / (1024**3)
print(f"Dostępna pamięć: {available_memory:.2f} GB")

# Dostosowanie rozmiaru kafelków na podstawie dostępnej pamięci
if memory_gb > available_memory * 0.8:
    tile_size = 1024
    print("Duże dane - używanie małych kafelków (1024px)")
else:
    tile_size = 2048
    print("Używanie standardowych kafelków (2048px)")

# Uruchomienie mozaikowania
mosaic_with_gauss_grid(
        input_folder,
        output_file,
        feather_px=40,
        tile_size=tile_size,
        cleanup_temp=True
)

Szacowanie wymagań pamięciowych...
Szacowany rozmiar danych: 0.13 GB
Wymiary: 2654x3102 pikseli
Dostępna pamięć: 11.22 GB
Używanie standardowych kafelków (2048px)
Katalog tymczasowy: /tmp/mosaic_temp_xic6x6ns
Znaleziono 26 plików do przetworzenia
Wyznaczanie globalnych granic...


Analizowanie granic: 100%|██████████| 25/25 [00:00<00:00, 399.81it/s]


Wymiary końcowej mozaiki: 2694x3142 pikseli
Szacowany rozmiar: 0.06 GB (float64)
Inicjalizacja plików tymczasowych...


Inicjalizacja plików temp: 100%|██████████| 4/4 [00:00<00:00, 13.76it/s]


Przetwarzanie kafelków...


...

Użycie pamięci: 11.3%


Przetwarzanie kafelków: 100%|██████████| 26/26 [00:00<00:00, 39.51it/s]


Użycie pamięci: 11.3%
Tworzenie finalnej mozaiki z blendowaniem...


Zapisywanie mozaiki: 100%|██████████| 2/2 [00:00<00:00,  2.76it/s]

Mozaika zapisana: Pila_outdoor_comfort.tif
Granice mozaiki: 343756.891, 580176.3482654289, 356823.45580114773, 595485.8059
Usuwanie plików tymczasowych...
